In [ ]:
import requests
import ipywidgets as widgets

In [ ]:
#dev api key + steamid

key = ""
steamid = ""

#custom methods
def handle_large_friend_lists(friend_ids):
    arr = [steamid]
    while len(friend_ids) > 0:
        count_of_ids = len(friend_ids)
        if count_of_ids > 100:
            arr.append(friend_ids[0:100])
            arr.delete(friend_ids[0:100])
        else:
            arr.append(friend_ids)
            break
    return arr

def fast_filter(name, arr):
    items = list(filter(lambda x: x["personaname"] == name, arr))
    if len(items) > 0:
        return items[0]
    
#urls
get_friends_url = "http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={key}&steamid={steamid}"
get_games_url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key={key}&steamid={steamid}&include_appinfo=true"
get_friend_details_url = "https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/?key={key}&steamids={ids}"

In [ ]:
friend_info = []
all_friends_response = requests.get(get_friends_url.format(key = key, steamid = steamid))
if all_friends_response.status_code == 200:  
    friend_ids = handle_large_friend_lists(list(map(lambda x: x["steamid"], all_friends_response.json()['friendslist']['friends'])))

for id_arr in friend_ids:
    get_friend_details_response = requests.get(get_friend_details_url.format(key = key, ids = id_arr))
    if get_friend_details_response.status_code == 200:
        friend_info.append(get_friend_details_response.json()["response"]["players"])

friend_info = [item for sublist in friend_info for item in sublist]

selected_friends = []
buttons = []
    
def handle_select(name):
    name = name["owner"].description
    if name in selected_friends:
        selected_friends.remove(name)
    else:
        selected_friends.append(name)
    
for x in list(map(lambda x: x["personaname"], friend_info)):
    button = widgets.ToggleButton(
        value=False,
        description=x,
        disabled=False,
        button_style='',
        icon='check'
    )
    button.observe(handle_select, 'value')
    buttons.append(button)

print("Select what friends to compare:")
widgets.VBox(buttons)

In [ ]:
friends_and_games = {}
if len(selected_friends) < 2:
    print("Please select at least 2 friends from above")
else:
    for name in selected_friends:
        friend = fast_filter(name, friend_info)
        games_response = requests.get(get_games_url.format(key = key, steamid = friend["steamid"]))
        if games_response.status_code == 200 and "games" in games_response.json()["response"]:
            friends_and_games[name] = games_response.json()["response"]["games"]

    missing_people = list(set(selected_friends) - set(friends_and_games.keys()))
    game_sets = []
    for name, games in friends_and_games.items():
        game_sets.append(set(list(map(lambda x: x["name"], games))))
    game_sets = list(game_sets)
    games = game_sets[0].intersection(*game_sets)
    if len(games) > 0:
        print(f"Here are all the games {list(friends_and_games.keys())} own:\n")
    print(games if (len(games) > 0) else "There are no overlapping games.")
    if len(missing_people) > 0:
        print(f"\n{missing_people} was not checked because they likely have a private profile.")